In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import re
train = pd.read_csv(r"C:\Users\raven\Downloads\sf-crime\train.csv\train.csv")
test  = pd.read_csv(r"C:\Users\raven\Downloads\sf-crime\test.csv\test.csv")
train1 = train.drop(['Descript', 'Resolution'], axis = 1)

In [2]:
train1 = train1.query('Y != 90')

In [3]:
# Case10 DayOfWeek, PdDistrictどちらもone-hot train1.ver

Dt = pd.to_datetime(train1['Dates'])
train1['Date'] = Dt.dt.date
train1['Year'] = Dt.dt.year
train1['Month'] = Dt.dt.month
train1['Day'] = Dt.dt.day
train1['Hour'] = Dt.dt.hour
train1.drop(['Dates'], axis = 1, inplace = True)
def to_season(month_num):
    season = 'winter'
    if 3<=month_num<=5:
        season = 'spring'
    elif 6<=month_num<=8:
        season = 'summer'
    elif 9<=month_num<=11:
        season = 'autumn'
    return season
train1['Season'] =Dt.dt.month.apply(to_season)

train1_DOW = pd.get_dummies(train1['DayOfWeek'])


Ad_dict = {'.* / .*':0, '.*':1}
train1['Address'] = train1['Address'].replace(Ad_dict,regex = True)

le = LabelEncoder()
train1['Category'] = le.fit_transform(train1['Category'])

train1_PD = pd.get_dummies(train1['PdDistrict'])

train1 = pd.concat([train1, train1_PD, train1_DOW], axis = 1)

train1.drop(['PdDistrict', 'DayOfWeek'], axis = 1, inplace=True)

train1.head()

,Category,Address,X,Y,Date,Year,Month,Day,Hour,Season,...,SOUTHERN,TARAVAL,TENDERLOIN,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,37,0,-122.425892,37.774599,2015-05-13,2015,5,13,23,spring,...,0,0,0,0,0,0,0,0,0,1
1,21,0,-122.425892,37.774599,2015-05-13,2015,5,13,23,spring,...,0,0,0,0,0,0,0,0,0,1
2,21,0,-122.424363,37.800414,2015-05-13,2015,5,13,23,spring,...,0,0,0,0,0,0,0,0,0,1
3,16,1,-122.426995,37.800873,2015-05-13,2015,5,13,23,spring,...,0,0,0,0,0,0,0,0,0,1
4,16,1,-122.438738,37.771541,2015-05-13,2015,5,13,23,spring,...,0,0,0,0,0,0,0,0,0,1


In [4]:
train1_SS = pd.get_dummies(train1['Season'])

train1 = pd.concat([train1, train1_SS], axis = 1)

train1.drop(['Season'], axis = 1, inplace=True)

In [5]:
# Case10 DayOfWeek, PdDistrictどちらもone-hot test.ver

Dt = pd.to_datetime(test['Dates'])
test['Date'] = Dt.dt.date
test['Year'] = Dt.dt.year
test['Month'] = Dt.dt.month
test['Day'] = Dt.dt.day
test['Hour'] = Dt.dt.hour
test.drop(['Dates'], axis = 1, inplace = True)
def to_season(month_num):
    season = 'winter'
    if 3<=month_num<=5:
        season = 'spring'
    elif 6<=month_num<=8:
        season = 'summer'
    elif 9<=month_num<=11:
        season = 'autumn'
    return season
test['Season'] =Dt.dt.month.apply(to_season)

test_DOW = pd.get_dummies(test['DayOfWeek'])


Ad_dict = {'.* / .*':0, '.*':1}
test['Address'] = test['Address'].replace(Ad_dict,regex = True)

test_PD = pd.get_dummies(test['PdDistrict'])

test = pd.concat([test, test_PD, test_DOW], axis = 1)

test.drop(['PdDistrict', 'DayOfWeek'], axis = 1, inplace=True)

test.head()

,Id,Address,X,Y,Date,Year,Month,Day,Hour,Season,...,SOUTHERN,TARAVAL,TENDERLOIN,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,1,-122.399588,37.735051,2015-05-10,2015,5,10,23,spring,...,0,0,0,0,0,0,1,0,0,0
1,1,0,-122.391523,37.732432,2015-05-10,2015,5,10,23,spring,...,0,0,0,0,0,0,1,0,0,0
2,2,1,-122.426002,37.792212,2015-05-10,2015,5,10,23,spring,...,0,0,0,0,0,0,1,0,0,0
3,3,1,-122.437394,37.721412,2015-05-10,2015,5,10,23,spring,...,0,0,0,0,0,0,1,0,0,0
4,4,1,-122.437394,37.721412,2015-05-10,2015,5,10,23,spring,...,0,0,0,0,0,0,1,0,0,0


In [6]:
test_SS = pd.get_dummies(test['Season'])

test = pd.concat([test, test_SS], axis = 1)

test.drop(['Season'], axis = 1, inplace=True)

In [7]:
cols = ['X', 'Y', 'Year', 'Month', 'Day', 'Hour',"spring","summer","autumn","winter", 'BAYVIEW', 'CENTRAL', 'INGLESIDE', 'MISSION', 'NORTHERN', 'PARK', 'RICHMOND', 'SOUTHERN', 'TARAVAL', 'TENDERLOIN', 'Sunday', 'Monday','Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
X = train1[cols]
Y = train1['Category']
X_test = test[cols]

X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.3, random_state = 1219)

In [8]:
model = XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

In [9]:
accuracy_score(y_val, y_pred)

0.28462575219727027

In [10]:
xgb_submission = model.predict(X_test)

In [11]:
import csv
pred = model.predict_proba(test[['X', 'Y', 'Year', 'Month', 'Day', 'Hour',"spring","summer","autumn","winter", 'BAYVIEW', 'CENTRAL', 'INGLESIDE', 'MISSION', 'NORTHERN', 'PARK', 'RICHMOND', 'SOUTHERN', 'TARAVAL', 'TENDERLOIN', 'Sunday', 'Monday','Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']])
submission = pd.DataFrame(columns=['Id'], data=test)
submission = pd.concat([submission, pd.DataFrame(pred, columns=["ARSON", "ASSAULT", "BAD CHECKS", "BRIBERY", "BURGLARY", "DISORDERLY CONDUCT",
"DRIVING UNDER THE INFLUENCE", "DRUG/NARCOTIC", "DRUNKENNESS", "EMBEZZLEMENT", "EXTORTION",
"FAMILY OFFENSES", "FORGERY/COUNTERFEITING", "FRAUD", "GAMBLING", "KIDNAPPING", "LARCENY/THEFT",
"LIQUOR LAWS", "LOITERING", "MISSING PERSON", "NON-CRIMINAL", "OTHER OFFENSES", "PORNOGRAPHY/OBSCENE MAT",
"PROSTITUTION", "RECOVERED VEHICLE", "ROBBERY", "RUNAWAY", "SECONDARY CODES", "SEX OFFENSES FORCIBLE",
"SEX OFFENSES NON FORCIBLE", "STOLEN PROPERTY", "SUICIDE", "SUSPICIOUS OCC", "TREA", "TRESPASS",
"VANDALISM", "VEHICLE THEFT", "WARRANTS", "WEAPON LAWS"])], axis=1)
submission.head()

,Id,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0,0.002871,0.132356,0.000116,0.003110,0.018311,0.000874,0.006959,0.013580,0.002902,...,0.000140,0.009638,0.001055,0.031041,8.207990e-07,0.003703,0.080462,0.153829,0.035057,0.026389
1,1,0.002762,0.097191,0.000022,0.000706,0.010818,0.000942,0.004910,0.021415,0.004751,...,0.000314,0.010390,0.001220,0.024300,7.137720e-06,0.003176,0.028913,0.079371,0.071524,0.025187
2,2,0.001929,0.043396,0.000024,0.000039,0.093177,0.000875,0.001387,0.013154,0.001730,...,0.000003,0.004288,0.000131,0.014578,3.160763e-07,0.011581,0.054616,0.107627,0.013580,0.002402
3,3,0.001626,0.104808,0.000027,0.009113,0.014778,0.001369,0.007732,0.013505,0.002807,...,0.000110,0.005521,0.000147,0.028649,5.404376e-07,0.001843,0.063521,0.205533,0.024685,0.020630
4,4,0.001626,0.104808,0.000027,0.009113,0.014778,0.001369,0.007732,0.013505,0.002807,...,0.000110,0.005521,0.000147,0.028649,5.404376e-07,0.001843,0.063521,0.205533,0.024685,0.020630


In [12]:
submission.to_csv('XGB_10.csv',index = False)